# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [1]:
from collections import defaultdict
from itertools import product

import numpy as np
from numpy.random import choice

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [2]:
def dna_to_rna(s):
    def f(x):
        acgt = list("ACGT")
        acgu = list("ACGT".replace('T', 'U'))
        return dict(zip(acgt, acgu))[x]
    return "".join(map(f, s))
    
if __name__ == '__main__':
    s = "AACGTGATTTC"
    assert dna_to_rna(s) == s.replace('T', 'U')
    print(dna_to_rna(s))

AACGUGAUUUC


### Idea of solution

fill in 

### Discussion

fill in 

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [3]:
sssss = '''UUU F 0.46 17.6 (714298)  UCU S 0.19 15.2 (618711)  UAU Y 0.44 12.2 (495699)  UGU C 0.46 10.6 (430311)
    UUC F 0.54 20.3 (824692)  UCC S 0.22 17.7 (718892)  UAC Y 0.56 15.3 (622407)  UGC C 0.54 12.6 (513028)
    UUA L 0.08  7.7 (311881)  UCA S 0.15 12.2 (496448)  UAA * 0.30  1.0 ( 40285)  UGA * 0.47  1.6 ( 63237)
    UUG L 0.13 12.9 (525688)  UCG S 0.05  4.4 (179419)  UAG * 0.24  0.8 ( 32109)  UGG W 1.00 13.2 (535595)

    CUU L 0.13 13.2 (536515)  CCU P 0.29 17.5 (713233)  CAU H 0.42 10.9 (441711)  CGU R 0.08  4.5 (184609)
    CUC L 0.20 19.6 (796638)  CCC P 0.32 19.8 (804620)  CAC H 0.58 15.1 (613713)  CGC R 0.18 10.4 (423516)
    CUA L 0.07  7.2 (290751)  CCA P 0.28 16.9 (688038)  CAA Q 0.27 12.3 (501911)  CGA R 0.11  6.2 (250760)
    CUG L 0.40 39.6 (1611801)  CCG P 0.11  6.9 (281570)  CAG Q 0.73 34.2 (1391973)  CGG R 0.20 11.4 (464485)

    AUU I 0.36 16.0 (650473)  ACU T 0.25 13.1 (533609)  AAU N 0.47 17.0 (689701)  AGU S 0.15 12.1 (493429)
    AUC I 0.47 20.8 (846466)  ACC T 0.36 18.9 (768147)  AAC N 0.53 19.1 (776603)  AGC S 0.24 19.5 (791383)
    AUA I 0.17  7.5 (304565)  ACA T 0.28 15.1 (614523)  AAA K 0.43 24.4 (993621)  AGA R 0.21 12.2 (494682)
    AUG M 1.00 22.0 (896005)  ACG T 0.11  6.1 (246105)  AAG K 0.57 31.9 (1295568)  AGG R 0.21 12.0 (486463)

    GUU V 0.18 11.0 (448607)  GCU A 0.27 18.4 (750096)  GAU D 0.46 21.8 (885429)  GGU G 0.16 10.8 (437126)
    GUC V 0.24 14.5 (588138)  GCC A 0.40 27.7 (1127679)  GAC D 0.54 25.1 (1020595)  GGC G 0.34 22.2 (903565)
    GUA V 0.12  7.1 (287712)  GCA A 0.23 15.8 (643471)  GAA E 0.42 29.0 (1177632)  GGA G 0.25 16.5 (669873)
    GUG V 0.46 28.1 (1143534)  GCG A 0.11  7.4 (299495)  GAG E 0.58 39.6 (1609975)  GGG G 0.25 16.5 (669768)'''

import re
import pandas as pd

m = re.findall(r'[A-Z]{3} [A-Z*]{1} [\d.]+ +[\d.]+ +\([ \d]+\)', sssss)
#print(m)
f = lambda x: x.replace('( ', '(').replace('(', '').replace(')', '')
m = [f(x) for x in m]
l = [list(map(lambda el: el.strip('()'), x.split())) for x in m]
#print(l)
#for x in l:
#    print(len(x), x)
df = pd.DataFrame(l, columns = ['codon', 'amino', 'fraction', 'freq', 'number'])
df = df[['codon', 'amino', 'fraction', 'freq', 'number']]
df.fraction = pd.to_numeric(df.fraction)
df.freq = pd.to_numeric(df.freq)
df.number = pd.to_numeric(df.number)
df = df.set_index('codon')
print(df[df.amino=='*'])
#print(df.head())
#print(df.dtypes)

def get_dict():
    return dict(zip(df.index, df.amino))
    
if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)

      amino  fraction  freq  number
codon                              
UAA       *      0.30   1.0   40285
UGA       *      0.47   1.6   63237
UAG       *      0.24   0.8   32109
{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}


### Idea of solution

fill in 

### Discussion

fill in 

#3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [4]:
#print(df.head())
#print(df.amino.unique(), len(df.amino.unique()))
#print(df.triplet.unique(), len(df.triplet.unique()))

def get_dict_list():
    d = dict()
    codon_to_aa = get_dict()
    for x in df.amino.unique():
        d[x] = []
    for x in df.index.unique():
        d[codon_to_aa[x]].append(x)
    return d
    
if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)

get_dict_list()['F']    

{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}


['UUU', 'UUC']

### Idea of solution

fill in 

### Discussion

fill in 

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [5]:
def rna_to_prot(s):
    acc = []
    l = list(s)
    while len(l) >= 3:
        acc.append("".join(l[:3]))
        l = l[3:]
    #print(acc)
    return "".join(list(map(lambda x: get_dict()[x], acc)))

def dna_to_prot(s):
    return rna_to_prot(dna_to_rna(s))

if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))

MISSTM*


### Idea of solution

fill in 

### Discussion

fill in 

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more **frequent** than others. These **frequencies** can be converted to **probabilities**. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called **codon adaptation**, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation **frequencies** in [**table**](http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html) and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function. Use the column "([number])" to estimate the probabilities, as the two preceding columns contain truncated values.  

In [6]:
df['probability'] = df.apply(lambda x: x.number / df[df.amino==x.amino].number.sum(), axis=1)
df.head()

,amino,fraction,freq,number,probability
codon,,,,,
UUU,F,0.46,17.6,714298,0.464134
UCU,S,0.19,15.2,618711,0.187586
UAU,Y,0.44,12.2,495699,0.443338
UGU,C,0.46,10.6,430311,0.456157
UUC,F,0.54,20.3,824692,0.535866


In [7]:
#print(df.loc['UAA', :], df.sum().number)
#print(df.isnull().sum())
#df = df.set_index('codon')

def get_probabability_dict():
    d = dict()
    for codon in df.index:
        aa = df.loc[codon,'amino']
        d[codon] = df.loc[codon,'number'] / df.groupby('amino').sum().loc[aa,'number']
    return d

print(df[df.amino=='*'])
#for x in df.amino.unique():
#    print(df[df.amino==x])

if __name__ == '__main__':
    codon_to_prob = get_probabability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))

      amino  fraction  freq  number  probability
codon                                           
UAA       *      0.30   1.0   40285     0.297019
UGA       *      0.47   1.6   63237     0.466243
UAG       *      0.24   0.8   32109     0.236738
AAA: 0.434049	AAC: 0.529633	AAG: 0.565951	AAU: 0.470367	ACA: 0.284188	ACC: 0.355232
ACG: 0.113812	ACU: 0.246769	AGA: 0.214658	AGC: 0.239938	AGG: 0.211091	AGU: 0.149602
AUA: 0.169062	AUC: 0.469866	AUG: 1.000000	AUU: 0.361072	CAA: 0.265017	CAC: 0.581485
CAG: 0.734983	CAU: 0.418515	CCA: 0.276603	CCC: 0.323470	CCG: 0.113196	CCU: 0.286731
CGA: 0.108812	CGC: 0.183777	CGG: 0.201554	CGU: 0.080108	CUA: 0.071380	CUC: 0.195577
CUG: 0.395702	CUU: 0.131716	GAA: 0.422453	GAC: 0.535458	GAG: 0.577547	GAU: 0.464542
GCA: 0.228121	GCC: 0.399781	GCG: 0.106176	GCU: 0.265922	GGA: 0.249922	GGC: 0.337109
GGG: 0.249882	GGU: 0.163087	GUA: 0.116577	GUC: 0.238306	GUG: 0.463346	GUU: 0.181770
UAA: 0.297019	UAC: 0.556662	UAG: 0.236738	UAU: 0.443338	UCA: 0.150517	UCC: 0.217960

### Idea of solution

fill in 

### Discussion

fill in 

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [8]:
class ProteinToMaxRNA:
    
    def __init__(self):
        pass

    def convert(self, s):
        ret = []
        prot2codons = get_dict_list()
        codon2prob = get_probabability_dict()
        for x in s:
            codons = prot2codons[x]
            l = [codon2prob[y] for y in codons]
            idx = np.argmax(l)
            codon = codons[idx]
            ret.append(codon)
        return "".join(ret)


if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA"))

CUGACCCCCAUCCAGAACAGAGCC


### Idea of solution

fill in 

### Discussion

fill in 

In [9]:
df[df.amino=='I']

,amino,fraction,freq,number,probability
codon,,,,,
AUU,I,0.36,16.0,650473,0.361072
AUC,I,0.47,20.8,846466,0.469866
AUA,I,0.17,7.5,304565,0.169062


Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [10]:
np.cumsum([1,2,3,4,5])

array([ 1,  3,  6, 10, 15])

In [11]:
def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    chars, l = list(dist.keys()), list(dist.values())
    x = np.random.uniform()
    l = np.cumsum(l)
    for i, v in enumerate(l):
        if x > v:
            continue
        return chars[i]
    #return next(iter(dist))

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))

G, T, G, A, T, T, A, A, A, T, T, T, T, G, G, C, C, C, T, T, T, G, G, G, C, T, C, T, A


### Idea of solution

fill in 

### Discussion

fill in 

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [12]:
class ProteinToRandomRNA(object):
    
    def __init__(self):
        pass

    def convert(self, s):
        ret = []
        prot2codons = get_dict_list()
        codon2prob = get_probabability_dict()
        for x in s:
            codons = prot2codons[x]
            l = [codon2prob[y] for y in codons]
            codon = random_event(dict(zip(codons, l)))
            ret.append(codon)
        return "".join(ret)
        
if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))

UUGACACCUAUACAGAACAGAGCC


### Idea of solution

fill in 

### Discussion

fill in 

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [13]:
# 01234567
# 012
#  123
#   234
#    345
#     456
#      567
from collections import Counter

def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    l = list(s)
    for i in range(len(s)-k+1):
        orig = dict()
        for x in set(list(s)):
            orig[x] = 0
        w = np.array(l[i:i+k])
        #u = np.unique(w, return_counts=True)
        #d = dict(zip(u[0], u[1]))
        orig.update(dict(Counter(w)))
        yield orig
    
if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC" #DNA
    for d in sliding_window(s, 4):
        print(d)

{'A': 0, 'C': 3, 'G': 0, 'T': 1}
{'A': 0, 'C': 3, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}
{'A': 0, 'C': 2, 'G': 0, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}


### Idea of solution

fill in 

### Discussion

fill in 

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [14]:
def context_list(s, k):
    d = dict()
    l = list(s)
    for i in range(len(l)-k):
        a = "".join(l[i:i+k])
        c = l[i+k]
        if a in d.keys():
            d[a] += c
        else:
            d[a] = c
    return d
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = context_list(s, k)
    print(d)

 # AT: GACCG
 # TG: AT
 # GA: TCT
 # TA: TG
 # TC: AG
 #    

{'AT': 'GACCG', 'TG': 'AT', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AG', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'GT': 'A'}


### Idea of solution

fill in

### Discussion

fill in

##11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [15]:
def f(s):
    d = Counter(s)
    total = sum(d.values())
    for k, v in d.items():
        d[k] /= total 
    return dict(d)

def context_probabilities(s, k):
    d = context_list(s, k)
    #print(d)
    for k, v in d.items():
        d[k] =  f(v)
    for k, v in d.items():
        for x in list("ACGT"):
            if not x in v.keys():
                v.update({x:0.})
        assert len(v) == 4, print(v)        
    return d
    
if __name__ == '__main__':
    for k in [0, 2]:
        s = "ATGATATCATCGACGATGTAG"
        d = context_probabilities(s, k)
        print(d)

{'': {'A': 0.3333333333333333, 'T': 0.2857142857142857, 'G': 0.23809523809523808, 'C': 0.14285714285714285}}
{'AT': {'G': 0.4, 'A': 0.2, 'C': 0.4, 'T': 0.0}, 'TG': {'A': 0.5, 'T': 0.5, 'C': 0.0, 'G': 0.0}, 'GA': {'T': 0.6666666666666666, 'C': 0.3333333333333333, 'A': 0.0, 'G': 0.0}, 'TA': {'T': 0.5, 'G': 0.5, 'A': 0.0, 'C': 0.0}, 'TC': {'A': 0.5, 'G': 0.5, 'C': 0.0, 'T': 0.0}, 'CA': {'T': 1.0, 'A': 0.0, 'C': 0.0, 'G': 0.0}, 'CG': {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}, 'AC': {'G': 1.0, 'A': 0.0, 'C': 0.0, 'T': 0.0}, 'GT': {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}}


### Idea of solution

fill in

### Discussion

fill in

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [16]:
class MarkovChain:
    
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def generate(self, n, seed=None):
        k = self.k
        kth = self.kth
        mer = "".join([random_event(self.zeroth) for _ in range(k)])
        if n < k:
            return mer[:n]
        key = mer
        while len(mer) < n:
            mer += random_event(kth[key])
            key = "".join(mer[-k:])
        return mer

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 0    
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))

### Idea of solution

fill in

### Discussion

fill in 

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use **pseudo counts**. That is, **all imaginable events are initialized to frequency count 1**.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [17]:
import itertools

def my_context_probabilities(s, k):

    def f(s):
        d = Counter(s+"ACGT")
        #print(d)
        total = sum(d.values())
        for k, v in d.items():
            d[k] /= total 
        return dict(d)

    d = context_list(s, k)
    #print(d)
    for k, v in d.items():
        d[k] =  f(v)
    for k, v in d.items():
        for x in list("ACGT"):
            if not x in v.keys():
                v.update({x:0.})
        assert len(v) == 4, print(v)        
    return d

def template_probabilities(k):
    d = dict()
    l = ["".join(x) for x in list(itertools.product('ACGT', repeat=k))]
    #print(l)
    for x in l:
        d[x] = dict(zip(list('ACGT'), [0.25]*len('ACGT')))
    #print(d)
    return d

def context_pseudo_probabilities(s, k):
    d = template_probabilities(k)
    d1 = my_context_probabilities(s, k)
    #print(d1)
    d.update(d1)
    #print(d)
    return d
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    #zeroth = context_pseudo_probabilities(s, 0)[""]
    #print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    
    #print("\n", MarkovChain(zeroth, kth, k).generate(20))        self.assertAlmostEqual(d["CT"]['C'], 0.250000, places=places)

AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'G': 0.4, 'A': 0.2, 'C': 0.2, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'G': 0.3333333333333333, 'A': 0.2222222222222222, 'C': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'T': 0.4, 'A': 0.2, 'C': 0.2, 'G': 0.2}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'T': 0.42857142857142855, 'C': 0.2857142857142857, 'A': 0.14285714285714285, 'G': 0.14285714285714285}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'T': 0.3333333333333333, 'G': 0.3333333333333333, 'A': 0.16666666666666666, 'C': 0.16666666666666666}
TC: {'A': 0.3333333333333333, 'G': 0.3333333333333333, 'C': 0.16666666666666666, 'T': 0.16666666666666666}
TG: {'A': 0.3333333333333333, 'T': 0.33333333333333

### Idea of solution

fill in 

### Discussion

fill in 

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [18]:
class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, s):
        k, zeroth, kth = self.k, self.zeroth, self.kth
        prob = 1.
        if len(s) < k:
            for i in range(len(s)):
                prob *= zeroth[s[i]]
                return prob
        else:
            for i in range(k):
                prob *= zeroth[s[i]]
            for i in range(len(s)-k):
                w = s[i:i+k]
                d = kth[w]
                c = s[i+k]
                #print(w, c)
                prob *= d[c]
            return prob

    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovProb(2, zeroth, kth)
    #s="ATGATATCATCGACGATGTAG"
    s='A'
    print(f"Probability of sequence {s} is {mc.probability(s)}")

Probability of sequence A is 0.32


### Idea of solution

fill in

### Discussion

fill in

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [19]:
import math

class MarkovLog(object):

    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth

    def log_probability(self, s):
        k, kth = self.k, self.kth
        prob = 0
        for i in range(k):
            #print(self.zeroth[s[i]])
            prob += math.log(self.zeroth[s[i]], 2)
        for i in range(len(s)-k):
            w = s[i:i+k]
            d = kth[w]
            c = s[i+k]
            #print(w, c)
            prob += math.log(d[c], 2)
        return prob
    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

Log probability of sequence ATGATATCATCGACGATGTAG is -31.717831515538315


### Idea of solution

fill in

### Discussion

fill in

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [20]:
def better_context_probabilities(s, k):
    kth = context_pseudo_probabilities(s, k)
    for k, v in kth.items():
        d = {'A':0.,'C':0.,'G':0.,'T':0.}
        d.update(v)
        kth[k] = d
    return dict(kth)

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    print(len(d))
    for k, v in d.items():
        assert len(v) == 4
    print("\n".join(f"{k}: {v}" for k, v in d.items()))
    print(d)
    d['CT']['C']

16
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666}
TG: {'A': 0.3333333333333333, 'C': 0.16666666666

### Idea of solution

fill in

### Discussion

fill in

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [21]:
class SimpleMarkovChain(object):
    def __init__(self, s, k):
        self.k = k
        self.s = s
        self.zeroth = list(better_context_probabilities(s, 0).values())[0]
        self.zeroth = list(self.zeroth.values())
        if k == 0:
            self.kth = self.zeroth
        else:
            self.kth = better_context_probabilities(s, k)
            for k, v in self.kth.items():
                self.kth[k] = list(v.values())
        #print(f'self.zeroth:{self.zeroth}')
        #print(f'self.kth:{self.kth}')

    def generate(self, n, seed=None):
        if seed != None:
            np.random.seed(seed)
        k, s = self.k, self.s
        mer = np.random.choice(list("ACGT"), k, p=self.zeroth)
        if k == 0:
            return "".join(np.random.choice(list("ACGT"), n, p=self.zeroth))
        if n < k:
            return "".join(mer[:n])
        mer = "".join(mer)
        key = mer
        while len(mer) < n:
            c = np.random.choice(list("ACGT"), 1, p=self.kth[key])
            mer += "".join(c)
            key = "".join(mer[-k:])
        return mer        
        
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 0
    for n in range(40):
        for k in range(4):
            print(f'n={n} k={k}')
            mc = SimpleMarkovChain(s, k)
            s2 = mc.generate(n, seed)
            assert len(s2)==n, print(f"Expected a string of length {n} from the 'generate' method!")
            print(s2)


n=0 k=0

n=0 k=1

n=0 k=2

n=0 k=3

n=1 k=0
G
n=1 k=1
G
n=1 k=2
G
n=1 k=3
G
n=2 k=0
GG
n=2 k=1
GG
n=2 k=2
GG
n=2 k=3
GG
n=3 k=0
GGG
n=3 k=1
GGC
n=3 k=2
GGG
n=3 k=3
GGG
n=4 k=0
GGGG
n=4 k=1
GGCG
n=4 k=2
GGGG
n=4 k=3
GGGG
n=5 k=0
GGGGC
n=5 k=1
GGCGA
n=5 k=2
GGGGC
n=5 k=3
GGGGC
n=6 k=0
GGGGCG
n=6 k=1
GGCGAT
n=6 k=2
GGGGCG
n=6 k=3
GGGGCG
n=7 k=0
GGGGCGC
n=7 k=1
GGCGATC
n=7 k=2
GGGGCGA
n=7 k=3
GGGGCGC
n=8 k=0
GGGGCGCT
n=8 k=1
GGCGATCT
n=8 k=2
GGGGCGAT
n=8 k=3
GGGGCGCT
n=9 k=0
GGGGCGCTT
n=9 k=1
GGCGATCTT
n=9 k=2
GGGGCGATT
n=9 k=3
GGGGCGCTT
n=10 k=0
GGGGCGCTTC
n=10 k=1
GGCGATCTTC
n=10 k=2
GGGGCGATTC
n=10 k=3
GGGGCGCTTC
n=11 k=0
GGGGCGCTTCT
n=11 k=1
GGCGATCTTCG
n=11 k=2
GGGGCGATTCG
n=11 k=3
GGGGCGCTTCT
n=12 k=0
GGGGCGCTTCTG
n=12 k=1
GGCGATCTTCGC
n=12 k=2
GGGGCGATTCGC
n=12 k=3
GGGGCGCTTCTG
n=13 k=0
GGGGCGCTTCTGG
n=13 k=1
GGCGATCTTCGCG
n=13 k=2
GGGGCGATTCGCG
n=13 k=3
GGGGCGCTTCTGG
n=14 k=0
GGGGCGCTTCTGGT
n=14 k=1
GGCGATCTTCGCGT
n=14 k=2
GGGGCGATTCGCGT
n=14 k=3
GGGGCGCTTCTGGT
n=15 k=0
GGGGCGCTTCT

### Idea of solution

fill in

### Discussion

fill in

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [22]:
import re

def kmer_index(s, k):
    d = dict()
    for kmer in ["".join(list(x)) for x in itertools.product('ACTG', repeat=k)]:
        v = [m.start() for m in re.finditer(kmer, s)]
        if v != []:
            d[kmer] = v
    return d

if __name__ == '__main__':
    k=2
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s)
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))

Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{'AC': [12], 'AT': [0, 3, 5, 8, 15], 'AG': [19], 'CA': [7], 'CG': [10, 13], 'TA': [4, 18], 'TC': [6, 9], 'TG': [1, 16], 'GA': [2, 11, 14], 'GT': [17]}


### Idea of solution

fill in

### Discussion

fill in

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [23]:
def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the proability of
    all 3-mers empirically based on the sequence
    """
    d = dict()
    for kmer in ["".join(list(x)) for x in itertools.product('ACGU', repeat=3)]:
        v = [m.start() for m in re.finditer(kmer, rna)]
        if v != []:
            d[kmer] = len(v)
        else:
            #continue
            d[kmer] = 0
    total = sum(d.values())
    for k, v in d.items():
        d[k] = v / total
    #print(sum(d.values()), d)
    return d

d = codon_probabilities("UACGACGUACCGGUUAACCGUU")
print(sum(d.values()), len(d), d)

1.0 64 {'AAA': 0.0, 'AAC': 0.05, 'AAG': 0.0, 'AAU': 0.0, 'ACA': 0.0, 'ACC': 0.1, 'ACG': 0.1, 'ACU': 0.0, 'AGA': 0.0, 'AGC': 0.0, 'AGG': 0.0, 'AGU': 0.0, 'AUA': 0.0, 'AUC': 0.0, 'AUG': 0.0, 'AUU': 0.0, 'CAA': 0.0, 'CAC': 0.0, 'CAG': 0.0, 'CAU': 0.0, 'CCA': 0.0, 'CCC': 0.0, 'CCG': 0.1, 'CCU': 0.0, 'CGA': 0.05, 'CGC': 0.0, 'CGG': 0.05, 'CGU': 0.1, 'CUA': 0.0, 'CUC': 0.0, 'CUG': 0.0, 'CUU': 0.0, 'GAA': 0.0, 'GAC': 0.05, 'GAG': 0.0, 'GAU': 0.0, 'GCA': 0.0, 'GCC': 0.0, 'GCG': 0.0, 'GCU': 0.0, 'GGA': 0.0, 'GGC': 0.0, 'GGG': 0.0, 'GGU': 0.05, 'GUA': 0.05, 'GUC': 0.0, 'GUG': 0.0, 'GUU': 0.1, 'UAA': 0.05, 'UAC': 0.1, 'UAG': 0.0, 'UAU': 0.0, 'UCA': 0.0, 'UCC': 0.0, 'UCG': 0.0, 'UCU': 0.0, 'UGA': 0.0, 'UGC': 0.0, 'UGG': 0.0, 'UGU': 0.0, 'UUA': 0.05, 'UUC': 0.0, 'UUG': 0.0, 'UUU': 0.0}


In [24]:
def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """
    #print(p)
    #print(q)

    if p == q:
        return 0.
    kpi = 0.
    unique_keys = lambda d: set([y for x in d.keys() for y in set(list(x))])
    for i in ["".join(list(x)) for x in itertools.product(unique_keys(p), repeat=len(list(set(p.keys()))[0]))]:
        #print(p[i], q[i])
        kpi += p[i] * math.log((p[i]+math.exp(-9)) / (q[i]), 2)
    return kpi

if __name__ == '__main__':

    if True:
        aas = list("*ACDEFGHIKLMNPQRSTVWY") # List of amino acids
        n = 10000
        
        # generate a random protein and some associated rna
        protein = "".join(choice(aas, n))
        rna = "".join(ProteinToRandomRNA().convert(protein))
        
        # Maybe check that converting back to protein results in the same sequence
        #print("".join(rna_to_prot(rna)))
        #print(protein)
        #assert "".join(rna_to_prot(rna)) == protein, "failed to convert back to protein from a generated rna"
        
        # Calculate codon probabilities of the rna sequence
        cp_predicted = codon_probabilities(rna) # placeholder call
        
        # Calculate codon probabilities based on the codon usage table
        #cp_orig = {"".join(codon): 0 for codon in product("ACGU", repeat=3)} # placeholder dict
        cp_orig = get_probabability_dict()
        
        # Create a completely random RNA sequence and get the codon probabilities
        cp_uniform = codon_probabilities("".join(choice(list("ACGU"), n))) # placeholder call
        
        #print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
        #print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
        #print()
        print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
        print(set(cp_uniform.values()))
        #print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
        #print()
        #print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
        #print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))

    if True:
        p = dict(zip("ACGT", [0.25]*4))
        assert kullback_leibler(p, p) == 0.0

    if True:
        p = dict(zip("ACGT", [1.0, 0.0, 0.0, 0.0]))
        q = dict(zip("ACGT", [0.25]*4))
        x = kullback_leibler(p, q)
        print(x)
        #ssert x == 2.0



d(original || uniform) = 97.48755725807439
{0.015700359158542843, 0.012211390456644433, 0.01590559261159569, 0.014161108260646486, 0.017650076962544895, 0.014366341713699333, 0.016110826064648538, 0.016316059517701386, 0.01806054386865059, 0.014571575166752181, 0.011082606464853772, 0.014776808619805027, 0.016521292970754234, 0.014982042072857875, 0.01323755772190867, 0.01672652642380708, 0.015187275525910723, 0.01693175987685993, 0.01539250897896357, 0.013648024628014366, 0.01734222678296562, 0.017136993329912777, 0.015597742432016419, 0.01210877373011801, 0.015802975885069265, 0.01754746023601847, 0.017752693689071317, 0.016008209338122113, 0.01426372498717291, 0.01621344279117496, 0.017957927142124165, 0.015495125705489995, 0.014468958440225757, 0.012724474089276553, 0.018163160595177013, 0.016418676244227808, 0.014674191893278605, 0.0172396100564392, 0.013134940995382247, 0.016829143150333504, 0.015084658799384299, 0.015289892252437147, 0.01375064135454079}
2.000178031727149


### Idea of solution

fill in

### Discussion

fill in

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [60]:
def get_stationary_distributions(transition):
    """
    The function get a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    w, v = np.linalg.eig(transition.T)
    #print(w, v)   
    def toprob(row):
        total = sum(row)
        return row / total
    x = [toprob(row) for row in [vec / val for val, vec in zip(w, v.T)]]
    return [x[1],x[3],x[0],x[2]]
    
if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    if False:
        print("\n".join(
            ", ".join(
                f"{pv:+.3f}"
                for pv in p) 
            for p in get_stationary_distributions(transition)))
    print(get_stationary_distributions(transition)[0])
    print(get_stationary_distributions(transition)[1])
    print(get_stationary_distributions(transition)[2])
    print(get_stationary_distributions(transition)[3])
#assert_allclose(distributions[0], [0.33333333, 0., 0.66666667,  0], atol=1e-04)
#assert_allclose(distributions[1], [ 0., 0.25, 0., 0.75], atol=1e-04)

[0.33333333 0.         0.66666667 0.        ]
[-0.    0.25 -0.    0.75]
[ 7.96131459e+15 -0.00000000e+00 -7.96131459e+15 -0.00000000e+00]
[ 0.00000000e+00  7.96131459e+15  0.00000000e+00 -7.96131459e+15]


### Idea of solution


### Discussion


21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [61]:
def kl_divergences(initial, transition):
    """
    Calculates the the Kullback-Leibler divergences between empirical distributions
    generated using a markov model seeded with an initial distributin and a transition 
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    return zip([1, 10, 100, 1000, 10000], np.random.rand(5))

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: # iterate on prefix lengths in order (1, 10, 100...)
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))

Transition probabilities are:
[[0.3  0.   0.7  0.  ]
 [0.   0.4  0.   0.6 ]
 [0.35 0.   0.65 0.  ]
 [0.   0.2  0.   0.8 ]]
Stationary distributions:
[[ 3.33333333e-01  0.00000000e+00  6.66666667e-01  0.00000000e+00]
 [-0.00000000e+00  2.50000000e-01 -0.00000000e+00  7.50000000e-01]
 [ 7.96131459e+15 -0.00000000e+00 -7.96131459e+15 -0.00000000e+00]
 [ 0.00000000e+00  7.96131459e+15  0.00000000e+00 -7.96131459e+15]]
Using [-0.00, 0.25, -0.00, 0.75] as initial distribution

KL divergence of stationary distribution prefix of length     1 is 0.91467483
KL divergence of stationary distribution prefix of length    10 is 0.59093059
KL divergence of stationary distribution prefix of length   100 is 0.12660189
KL divergence of stationary distribution prefix of length  1000 is 0.55871492
KL divergence of stationary distribution prefix of length 10000 is 0.26954798


### Idea of solution

fill in

### Discussion
fill in

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [ ]:
def main(transition, equilibrium_distribution):
    vals = list(zip(np.random.rand(10), np.random.rand(10, 4) - 0.5))
    return zip(np.random.rand(2, 4) - 0.5, 
               [vals[:5], vals[5:]])


if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")
    stationary_distributions = get_stationary_distributions(transition)
    # Uncomment the below line to check that there actually is only one stationary distribution
    # assert len(stationary_distributions) == 1
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)
    for initial_distribution, results in main(transition, equilibrium_distribution):
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        print("\n".join("{:.11f}   {}".format(di, kl) for di, kl in results))

### Idea of solution

fill in

### Discussion
fill in